# Learning the basics with OpenAI

For some time now, we've been using tools like ChatGPT and CoPilot for day-to-day tasks, but mostly through the conversational AI chatbots they provide. We've used them for help with drafting emails and providing type-ahead coding assistance. We haven't, however, gone a step further and integrated them into a development project. Honestly, we weren't sure where to start. Looking into the available options, you quickly run into a dozen new concepts, from vector stores to agents, and different SDKs that all seem to solve similar problems.

We want to document our learning as we go, and hopefully provide a useful resource for others who are also looking to get started. We'll start with the basics, using the OpenAI API directly, and then move on to more complex scenarios.

## Setting up your environment

We assume you've already followed the README instructions to set up your Python environment. If not, please do so now.

To start, we'll use OpenAI, so you'll need an API key. You can get one by signing up at [platform.openai.com](https://platform.openai.com). Once you have your key, store it in a `.envrc` file in the root of your project:

```sh
export OPENAI_API_KEY="<your_api_key_here>"
```

In [2]:
# VS Code's Jupyter extension doesn't support loading .envrc, so if you're using VS Code, we load it here.

from io import StringIO
from pathlib import Path

from dotenv import load_dotenv

envrc = Path("../.envrc")
stream = StringIO()
[stream.write(f"{line}\n") for line in envrc.read_text().splitlines() if line.startswith("export")]
stream.seek(0)

load_dotenv(stream=stream)

True

## Raw JSON Response using curl

Let's try something simple: ask for a short kid-friendly joke. We'll use `curl` to make a request to the OpenAI API directly, specifically the newer [Responses API](https://platform.openai.com/docs/api-reference/responses/create?lang=curl). This way, we can see the raw JSON response from the model.

In [15]:
%%bash --out curl_response

curl https://api.openai.com/v1/responses -s \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-5-nano",
    "input": "Tell me a silly joke for a kid."
  }'


In [ ]:
import json

from rich import print as rich_print

data = json.loads(curl_response)  # noqa
rich_print(data)

{
    'id': 'resp_0de7a53317c5803d0068c95c98dfa481959606d7469de872d3',
    'object': 'response',
    'created_at': 1758026904,
    'status': 'completed',
    'background': False,
    'error': None,
    'incomplete_details': None,
    'instructions': None,
    'max_output_tokens': None,
    'max_tool_calls': None,
    'model': 'gpt-5-nano-2025-08-07',
    'output': [
        {'id': 'rs_0de7a53317c5803d0068c95c99e4148195b97506d026694bc2', 'type': 'reasoning', 'summary': []},
        {
            'id': 'msg_0de7a53317c5803d0068c95c9bce0081958c3ec24e0cf723dc',
            'type': 'message',
            'status': 'completed',
            'content': [
                {
                    'type': 'output_text',
                    'annotations': [],
                    'logprobs': [],
                    'text': 'Why did the scarecrow win an award? Because he was outstanding in his field! Want 
another silly joke?'
                }
            ],
            'role': 'assistant'
        }
    ],
    'parallel_tool_calls': True,
    'previous_response_id': None,
    'prompt_cache_key': None,
    'reasoning': {'effort': 'medium', 'summary': None},
    'safety_identifier': None,
    'service_tier': 'default',
    'store': True,
    'temperature': 1.0,
    'text': {'format': {'type': 'text'}, 'verbosity': 'medium'},
    'tool_choice': 'auto',
    'tools': [],
    'top_logprobs': 0,
    'top_p': 1.0,
    'truncation': 'disabled',
    'usage': {
        'input_tokens': 15,
        'input_tokens_details': {'cached_tokens': 0},
        'output_tokens': 348,
        'output_tokens_details': {'reasoning_tokens': 320},
        'total_tokens': 363
    },
    'user': None,
    'metadata': {}
}

There's a lot in the JSON, but we can see the actual text response in `output[1].content`:

In [20]:
rich_print(data["output"][1]["content"])

[
    {
        'type': 'output_text',
        'annotations': [],
        'logprobs': [],
        'text': 'Why did the scarecrow win an award? Because he was outstanding in his field! Want another silly 
joke?'
    }
]

That's easy enough! We got a response back from the model, and can see how to extract the actual text from the JSON.

## Using the Python SDK

The same can be done using Python, again using the Responses API:

In [21]:
import openai

client = openai.Client()

In [22]:
response = client.responses.create(
    model="gpt-5-nano",
    input="Tell me a silly joke for a kid.",
    reasoning={"effort": "minimal"},
)
rich_print(response)

Response(
    id='resp_09d2579ed6bd3f9c0068c95cb4e5408192bea40f7f85aaaa95',
    created_at=1758026932.0,
    error=None,
    incomplete_details=None,
    instructions=None,
    metadata={},
    model='gpt-5-nano-2025-08-07',
    object='response',
    output=[
        ResponseReasoningItem(
            id='rs_09d2579ed6bd3f9c0068c95cb561d88192b98f08d6bd415a72',
            summary=[],
            type='reasoning',
            content=None,
            encrypted_content=None,
            status=None
        ),
        ResponseOutputMessage(
            id='msg_09d2579ed6bd3f9c0068c95cb574408192840fcf2ad221edce',
            content=[
                ResponseOutputText(
                    annotations=[],
                    text="Sure! Here’s a goofy one:\n\nWhy did the banana go to the doctor?\n\nBecause it wasn't 
peeling well!",
                    type='output_text',
                    logprobs=[]
                )
            ],
            role='assistant',
            status='completed',
            type='message'
        )
    ],
    parallel_tool_calls=True,
    temperature=1.0,
    tool_choice='auto',
    tools=[],
    top_p=1.0,
    background=False,
    conversation=None,
    max_output_tokens=None,
    max_tool_calls=None,
    previous_response_id=None,
    prompt=None,
    prompt_cache_key=None,
    reasoning=Reasoning(effort='minimal', generate_summary=None, summary=None),
    safety_identifier=None,
    service_tier='default',
    status='completed',
    text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'),
    top_logprobs=0,
    truncation='disabled',
    usage=ResponseUsage(
        input_tokens=15,
        input_tokens_details=InputTokensDetails(cached_tokens=0),
        output_tokens=29,
        output_tokens_details=OutputTokensDetails(reasoning_tokens=0),
        total_tokens=44
    ),
    user=None,
    store=True
)

As we can see, the response is very similar to what we got using `curl` and the API is quite straightforward to use.

# Token probabilities

We can also ask for the top token probabilities for each token in the response. This can be useful for understanding how the model works and how confident it is in its responses. I've always heard that LLMs are just "guessing" the next word based on probabilities, so this should be interesting to see.

OpenAI makes this easy to do by adding the `top_logprobs` parameter to our request:

In [37]:
response = client.responses.create(
    model="gpt-4",
    input="Tell me a silly joke for a kid in 5 words or less.",
    top_logprobs=5,
    include=["message.output_text.logprobs"],
)
rich_print(response.output[0].content[0])

ResponseOutputText(
    annotations=[],
    text="Why don't skeletons fight? No guts!",
    type='output_text',
    logprobs=[
        Logprob(
            token='Why',
            bytes=[87, 104, 121],
            logprob=-0.181642,
            top_logprobs=[
                LogprobTopLogprob(token='Why', bytes=[87, 104, 121], logprob=-0.181642),
                LogprobTopLogprob(token='"Why', bytes=[34, 87, 104, 121], logprob=-1.82203),
                LogprobTopLogprob(token='"', bytes=[34], logprob=-5.511552),
                LogprobTopLogprob(token='"W', bytes=[34, 87], logprob=-9.074697),
                LogprobTopLogprob(token='"What', bytes=[34, 87, 104, 97, 116], logprob=-10.026135)
            ]
        ),
        Logprob(
            token=' don',
            bytes=[32, 100, 111, 110],
            logprob=-0.002558,
            top_logprobs=[
                LogprobTopLogprob(token=' don', bytes=[32, 100, 111, 110], logprob=-0.002558),
                LogprobTopLogprob(token=' did', bytes=[32, 100, 105, 100], logprob=-6.899607),
                LogprobTopLogprob(token=' was', bytes=[32, 119, 97, 115], logprob=-6.964232),
                LogprobTopLogprob(token=' can', bytes=[32, 99, 97, 110], logprob=-8.24675),
                LogprobTopLogprob(token=' are', bytes=[32, 97, 114, 101], logprob=-9.430046)
            ]
        ),
        Logprob(
            token="'t",
            bytes=[39, 116],
            logprob=-0.001991,
            top_logprobs=[
                LogprobTopLogprob(token="'t", bytes=[39, 116], logprob=-0.001991),
                LogprobTopLogprob(token='’t', bytes=[226, 128, 153, 116], logprob=-6.220146),
                LogprobTopLogprob(token="'", bytes=[39], logprob=-16.725348),
                LogprobTopLogprob(token='ut', bytes=[117, 116], logprob=-17.867268),
                LogprobTopLogprob(token='key', bytes=[107, 101, 121], logprob=-18.265741)
            ]
        ),
        Logprob(
            token=' skeletons',
            bytes=[32, 115, 107, 101, 108, 101, 116, 111, 110, 115],
            logprob=-2.035915,
            top_logprobs=[
                LogprobTopLogprob(
                    token=' scientists',
                    bytes=[32, 115, 99, 105, 101, 110, 116, 105, 115, 116, 115],
                    logprob=-0.490537
                ),
                LogprobTopLogprob(token=' eggs', bytes=[32, 101, 103, 103, 115], logprob=-1.947259),
                LogprobTopLogprob(
                    token=' skeletons',
                    bytes=[32, 115, 107, 101, 108, 101, 116, 111, 110, 115],
                    logprob=-2.035915
                ),
                LogprobTopLogprob(
                    token=' elephants',
                    bytes=[32, 101, 108, 101, 112, 104, 97, 110, 116, 115],
                    logprob=-2.790405
                ),
                LogprobTopLogprob(token=' we', bytes=[32, 119, 101], logprob=-4.12195)
            ]
        ),
        Logprob(
            token=' fight',
            bytes=[32, 102, 105, 103, 104, 116],
            logprob=-1e-05,
            top_logprobs=[
                LogprobTopLogprob(token=' fight', bytes=[32, 102, 105, 103, 104, 116], logprob=-1e-05),
                LogprobTopLogprob(token=' pick', bytes=[32, 112, 105, 99, 107], logprob=-12.477577),
                LogprobTopLogprob(token=' start', bytes=[32, 115, 116, 97, 114, 116], logprob=-13.164175),
                LogprobTopLogprob(token=' fights', bytes=[32, 102, 105, 103, 104, 116, 115], logprob=-14.109425),
                LogprobTopLogprob(token=' watch', bytes=[32, 119, 97, 116, 99, 104], logprob=-14.984205)
            ]
        ),
        Logprob(
            token='?',
            bytes=[63],
            logprob=-0.032937,
            top_logprobs=[
                LogprobTopLogprob(token='?', bytes=[63], logprob=-0.032937),
                LogprobTopLogprob(token='?\n\n', bytes=[63, 10, 10], logprob=-4.134192),
                LogprobTopLogprob(token=' ea

This is cool. We can see the tokens that make up the response, along with the top tokens that the model considered at each step, and their associated probabilities. Let's take this a step further and put this data into a Pandas DataFrame for easier analysis:

In [ ]:
import pandas as pd

rows = []
# Loop through the response probabilities
for prob in response.output[0].content[0].logprobs:
    # Loop through each token's top logprobs
    for top_logprob in prob.top_logprobs:
        rows.append(
            {
                "token": prob.token,
                "top_logprob_token": top_logprob.token,
                "top_logprob": top_logprob.logprob,
            }
        )

pd.DataFrame(rows)

,token,top_logprob_token,top_logprob
0,Why,Why,-0.181642
1,Why,"""Why",-1.822030
2,Why,"""",-5.511552
3,Why,"""W",-9.074697
4,Why,"""What",-10.026135
5,don,don,-0.002558
6,don,did,-6.899607
7,don,was,-6.964232
8,don,can,-8.246750
9,don,are,-9.430046


Here we can see the top tokens the model considered at each step. OpenAI provides a helpful [cookbook on logprobs](https://cookbook.openai.com/examples/using_logprobs):

> Log probabilities of output tokens indicate the likelihood of each token occurring in the sequence given the context. To simplify, a logprob is `log(p)`, where `p` = probability of a token occurring at a specific position based on the previous tokens in the context. Some key points about `logprobs`:
> * Higher log probabilities suggest a higher likelihood of the token in that context. This allows users to gauge the model's confidence in its output or explore alternative responses the model considered.
> * Logprob can be any negative number or `0.0`. `0.0` corresponds to 100% probability.

I found this helpful to see how the model is "thinking" as it generates a response. We can see that for some tokens, the model was very confident (e.g., "fight"), while for others, it was less so (e.g., "watch"). This gives us some insight into how the model is generating its responses.

## Next steps

In summary, I found it helpful to see the raw JSON response from the model, to understand how to extract the actual text response. Additionally, seeing the token probabilities helps show me how the model is "thinking" as it generates text.

Next up, function calling!